# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
!pip install tabulate


In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from tabulate import tabulate


#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
    

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print(len(full_data_rows_list))
# First few entries of data
print(full_data_rows_list[:5])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88'], ['Building 429', 'Logged In', 'Mohammad', 'M', '2', 'Rodriguez', '300.61669', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Majesty (LP Version)', '200', '1.54328E+12', '88'], ["The B-52's", 'Logged In', 'Gianna', 'F', '0', 'Jones', '321.54077', 'free', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54087E+12', '107', 'Love Shack', '200', '1.54328E+12', '38'], ['Die Mooskirc

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [6]:
# Listing data columns with index for easy retrieving
# index, column_name
# ==================
# 0: artist_name
# 1: first_name
# 2: gender
# 3: item_in_session
# 4: last_name
# 5: song_length
# 6: level
# 7: user_location
# 8: session_id
# 9: song_title
# 10: user_id

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. 

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [10]:
# Create Table if not exists
query = "CREATE TABLE IF NOT EXISTS streaming_song_info "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, song_length float, PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

#### Justification
For table <code>streaming_song_info</code>, the <code>session_id</code> is used as a partition key and <code>item_in_session</code> is used as clustering column to help make up a unique key and the queries will filter by these columns.

In [11]:
# Read the data file and insert into streaming_song_info
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO streaming_song_info (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        

#### SELECT query to verify that the data have been inserted into table correctly


In [12]:
query = "SELECT artist_name, song_title, song_length FROM streaming_song_info WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
"""for row in rows:
    print (row.artist_name, row.song_title, row.song_length, sep=' | ')"""


df = pd.DataFrame(rows._current_rows)
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+---------------+---------------------------------+---------------+
|    | artist_name   | song_title                      |   song_length |
|----+---------------+---------------------------------+---------------|
|  0 | Faithless     | Music Matters (Mark Knight Dub) |       495.307 |
+----+---------------+---------------------------------+---------------+


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


In [13]:
# Create Table if not exists
query = "CREATE TABLE IF NOT EXISTS streaming_song_usersession_info "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Justification
For table <code>streaming_song_usersession_info</code>, the <code>user_id</code> and <code>session_id</code> are used as composite partition key because the queries will filter by these columns and <code>item_in_session</code> is used as clustering column to help make up a unique key.


In [14]:
# Read the data file and insert into streaming_song_usersession_info
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO streaming_song_usersession_info (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### SELECT query to verify that the data have been inserted into table correctly


In [15]:
query = "SELECT artist_name, song_title, first_name, last_name FROM streaming_song_usersession_info WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
"""for row in rows:
    print (row.artist_name, row.song_title, row.first_name, row.last_name, sep=' | ')"""
df = pd.DataFrame(rows._current_rows)
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+-------------------+------------------------------------------------------+--------------+-------------+
|    | artist_name       | song_title                                           | first_name   | last_name   |
|----+-------------------+------------------------------------------------------+--------------+-------------|
|  0 | Down To The Bone  | Keep On Keepin' On                                   | Sylvie       | Cruz        |
|  1 | Three Drives      | Greece 2000                                          | Sylvie       | Cruz        |
|  2 | Sebastien Tellier | Kilometer                                            | Sylvie       | Cruz        |
|  3 | Lonnie Gordon     | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie       | Cruz        |
+----+-------------------+------------------------------------------------------+--------------+-------------+


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
# Create Table if not exists
query = "CREATE TABLE IF NOT EXISTS streaming_song_users "
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY(song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

#### Justification
For table <code>streaming_song_users</code>, the <code>song_title</code> is used as a partition key and <code>user_id</code> is used as clustering column to help make up a unique key and the queries will filter by these columns.


In [17]:
# Read the data file and insert into streaming_song_usersession_info
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO streaming_song_users (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        

#### SELECT query to verify that the data have been inserted into each table

In [18]:
query = "SELECT first_name, last_name FROM streaming_song_users WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
"""for row in rows:
    print (row.first_name, row.last_name, sep=' | ')"""
df = pd.DataFrame(rows._current_rows)
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+--------------+-------------+
|    | first_name   | last_name   |
|----+--------------+-------------|
|  0 | Jacqueline   | Lynch       |
|  1 | Tegan        | Levine      |
|  2 | Sara         | Johnson     |
+----+--------------+-------------+


### Drop the tables before closing out the sessions

In [19]:
## Drop the table before closing out the sessions
try:
    session.execute("drop table if exists streaming_song_info")
    session.execute("drop table if exists streaming_song_usersession_info")
    session.execute("drop table if exists streaming_song_users")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()
